**Training Bert Baseline**

In [ ]:
# !pip3 install transformers
# !pip3 install --upgrade pandas==1.3.4

In [1]:
import pandas as pd
import torch
from transformers import BertModel
from transformers import BertTokenizer

In [2]:
opted_data = pd.read_pickle('datasets/Opted/opted.pkl')
wordnet_data = pd.read_pickle('datasets/WordNet/wordnet.pkl')
combined_data = pd.concat([opted_data, wordnet_data])

In [3]:
# ## toy data
# combined_data = combined_data[0:1000]

In [4]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 4 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in combined_data['definition']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences. Max length is 291 but most are not that long
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/apps/python3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [8]:
torch.save(input_ids, 'input_ids')
torch.save(attention_masks, 'attention_masks')

In [9]:
# All 400000 embeddings, each lengh 60
from gensim.models import FastText
Fastext_model = FastText.load("datasets/fastext_embeddings")

In [10]:
import numpy as np
labels = []
count = 0
for word in combined_data['word']:
    label = Fastext_model.wv[word]
    labels.append(label)
labels = np.array(labels)
labels = torch.tensor(labels)

In [11]:
print('Original: ', combined_data['definition'].iloc[0])
print('Token IDs:', input_ids[0])
print('Embedding: ', labels[0])

Original:  the first letter of the english and of many other alphabets. the capital a of the alphabets of middle and western europe, as also the small letter (a), besides the forms in italic, black letter, etc., are all descended from the old latin a, which was borrowed from the greek alpha, of the same form; and this was made from the first letter (/) of the phoenician alphabet, the equivalent of the hebrew aleph, and itself from the egyptian origin. the aleph was a consonant letter, with a guttural breath sound that was not an element of greek articulation; and the greeks took it to represent their vowel alpha with the a sound, the phoenician alphabet having no vowel symbols.
Token IDs: tensor([  101,  1996,  2034,  3661,  1997,  1996,  2394,  1998,  1997,  2116,
         2060, 12440,  2015,  1012,  1996,  3007,  1037,  1997,  1996, 12440,
         2015,  1997,  2690,  1998,  2530,  2885,  1010,  2004,  2036,  1996,
         2235,  3661,  1006,  1037,  1007,  1010,  4661,  1996,  359

In [12]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(224))

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

233,436 training samples
58,360 validation samples


In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(224)), # Select batches pseudo randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [14]:
## Bert hidden layer size = 768
## max sequence length = 200 

In [15]:
bert_layer = BertModel.from_pretrained('bert-base-uncased')
bert_layer.cuda()
model = torch.nn.Sequential(
        torch.nn.Linear(bert_layer.config.hidden_size * 200, 1024), ## gradual dimensionality reduction
        torch.nn.ReLU(), 
        torch.nn.LayerNorm(1024),
        torch.nn.Linear(1024, 256),
        torch.nn.ReLU(), 
        torch.nn.LayerNorm(256),
        torch.nn.Linear(256, 60),
        torch.nn.Softmax())
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sequential(
  (0): Linear(in_features=153600, out_features=1024, bias=True)
  (1): ReLU()
  (2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=1024, out_features=256, bias=True)
  (4): ReLU()
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): Linear(in_features=256, out_features=60, bias=True)
  (7): Softmax(dim=None)
)

In [16]:
from transformers import AdamW, BertConfig

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

2022-02-27 04:49:09.636460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/apps/python3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [18]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [21]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 224

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Save model
            torch.save(model.state_dict(), "model.params.e")

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = bert_layer(b_input_ids)
        outputs = outputs[0].view(outputs[0].shape[0], 153600) # 153600 = 768 * 200
        outputs = model(outputs)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end.
        loss = loss_fn(outputs, b_labels.float())
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.double().backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    top_count = 0
    top_10_count = 0
    top_100_count = 0
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = bert_layer(b_input_ids)
            outputs = outputs[0].view(outputs[0].shape[0], 153600)
            outputs = model(outputs)
            loss = loss_fn(outputs, b_labels.float())
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        for i in range(outputs.shape[0]): # batch_size
            top_100 = Fastext_model.wv.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=100)
            top_10 = Fastext_model.wv.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=10)[:10]
            actual = Fastext_model.wv.most_similar(positive=[b_labels[i].cpu().numpy().astype(np.float32)], topn=1)[0][0]
            top_count += actual == top_10[0][0]
            top_10_count += actual in [pairs[0] for pairs in top_10]
            top_100_count += actual in [pairs[0] for pairs in top_100]
        print(top_count, top_10_count, top_100_count)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = top_count / len(validation_dataloader)
    print("  Top 1 Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_10_accuracy = top_10_count / len(validation_dataloader)
    print("  Top 10 Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_100_accuracy = top_100_count / len(validation_dataloader)
    print("  Top 100 Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Top 1 Accur.': avg_val_accuracy,
            'Valid. Top 10 Accur.': avg_val_10_accuracy,
            'Valid. Top 100 Accur.': avg_val_100_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...


/apps/python3/lib/python3.7/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


  Batch    50  of  14,590.    Elapsed: 0:00:27.
  Batch   100  of  14,590.    Elapsed: 0:00:59.
  Batch   150  of  14,590.    Elapsed: 0:01:30.
  Batch   200  of  14,590.    Elapsed: 0:02:02.
  Batch   250  of  14,590.    Elapsed: 0:02:34.
  Batch   300  of  14,590.    Elapsed: 0:03:05.
  Batch   350  of  14,590.    Elapsed: 0:03:36.
  Batch   400  of  14,590.    Elapsed: 0:04:08.
  Batch   450  of  14,590.    Elapsed: 0:04:40.
  Batch   500  of  14,590.    Elapsed: 0:05:11.
  Batch   550  of  14,590.    Elapsed: 0:05:43.
  Batch   600  of  14,590.    Elapsed: 0:06:15.
  Batch   650  of  14,590.    Elapsed: 0:06:47.
  Batch   700  of  14,590.    Elapsed: 0:07:19.
  Batch   750  of  14,590.    Elapsed: 0:07:50.
  Batch   800  of  14,590.    Elapsed: 0:08:21.
  Batch   850  of  14,590.    Elapsed: 0:08:53.
  Batch   900  of  14,590.    Elapsed: 0:09:24.
  Batch   950  of  14,590.    Elapsed: 0:09:56.
  Batch 1,000  of  14,590.    Elapsed: 0:10:27.
  Batch 1,050  of  14,590.    Elapsed: 0

In [27]:
top_count = 0
top_10_count = 0
top_100_count = 0

for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = bert_layer(b_input_ids)
            outputs = outputs[0].view(outputs[0].shape[0], 153600)
            outputs = model(outputs)
            loss = loss_fn(outputs, b_labels.float())
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        for i in range(outputs.shape[0]): # batch_size
            top_100 = Fastext_model.wv.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=100)
            top_10 = Fastext_model.wv.most_similar(positive=[outputs[i].cpu().numpy().astype(np.float32)], topn=10)[:10]
            actual = Fastext_model.wv.most_similar(positive=[b_labels[i].cpu().numpy().astype(np.float32)], topn=1)[0][0]
            top_count += actual == top_10[0][0]
            top_10_count += actual in [pairs[0] for pairs in top_10]
            top_100_count += actual in [pairs[0] for pairs in top_100]
            #if (i%1000 == 0):
print(top_count, top_10_count, top_100_count)

# Report the final accuracy for this validation run.
avg_val_accuracy = top_count / len(validation_dataloader)
print("  Top 1 Accuracy: {0:.2f}".format(avg_val_accuracy))
avg_val_10_accuracy = top_10_count / len(validation_dataloader)
print("  Top 10 Accuracy: {0:.2f}".format(avg_val_accuracy))
avg_val_100_accuracy = top_100_count / len(validation_dataloader)
print("  Top 100 Accuracy: {0:.2f}".format(avg_val_accuracy))

# Calculate the average loss over all of the batches.
avg_val_loss = total_eval_loss / len(validation_dataloader)

# Measure how long the validation run took.
validation_time = format_time(time.time() - t0)

print("  Validation Loss: {0:.2f}".format(avg_val_loss))
print("  Validation took: {:}".format(validation_time))


4 18 294
  Top 1 Accuracy: 0.00
  Top 10 Accuracy: 0.00
  Top 100 Accuracy: 0.00
  Validation Loss: 4279.50
  Validation took: 6:07:31


In [28]:
len(validation_dataloader)

3648

In [32]:
100 * (top_count / len(validation_dataloader))

0.10964912280701754

In [33]:
100 * (top_10_count / len(validation_dataloader))

0.4934210526315789

In [34]:
100 *( top_100_count / len(validation_dataloader))

8.05921052631579